In [11]:
import re
import pandas as pd # CSV file I/O (pd.read_csv)
from nltk.corpus import stopwords

import copy
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [12]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\News data (Completed) (1) (9).xlsx")
news.head()

,Title,Body,Category
0,Adani Ports Shares Jump Nearly 6% as Buyback O...,"August 26, 2019, 4:25 PM IST\nFor Representati...",Buyback of shares - Alert me when any of these...
1,Adani Ports buyback to open on September 6; sh...,Adani Ports and Special Economic Zone\ngained ...,Buyback of shares - Alert me when any of these...
2,Adani Ports Q1 net profit jumps 47.50%,Adani Ports and Special Economic Zone (APSEZ) ...,"Earnings, Stock Performance or Dividends - Ale..."
3,Adani Ports keen on building port at Nagarjuna...,Adani Ports and Special Economic Zone Ltd (APS...,New Geography - Alert me when a company moves ...
4,Adani Ports Q1 result: Adani Ports Q1 PAT jump...,Adani Ports\non Wednesday said its consolidate...,"Earnings, Stock Performance or Dividends - Ale..."


In [107]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_words.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
    stop_words.union(['monday', 'tuesday', 'wednesday', 'thursday','friday','saturday','sunday'])
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w.lower() not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [108]:
stop_words1 = set(stopwords.words('english'))
stop_words1.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
print(stop_words1)

{'but', 'same', 'won', 'his', 'are', 'how', "needn't", 'if', 'so', "aren't", "wouldn't", 't', 're', 'out', 'such', 'both', 'needn', 'this', 'yourselves', 'during', 'where', 'through', "it's", 'a', 'hers', 'be', 'him', 'yourself', 'her', 'herself', 'into', "she's", 'then', 'an', 'will', 'weren', 'own', 'it', 'been', 'its', 'has', 'other', "shouldn't", 'don', "haven't", 'mustn', 'couldn', 'and', 'after', 'who', 'for', 'those', 'you', 'he', 'do', 'at', 'ma', 'off', 'was', 'all', 'nor', 'too', "don't", "you're", 'can', 'have', 'am', 'didn', 'mightn', 'itself', 'should', 'few', 'your', 'more', 'll', 'before', 'below', "weren't", "won't", 'doesn', 'doing', 'no', 've', 'my', 'we', 'ours', "that'll", 'on', 'here', 'their', 'me', 'very', "you'd", 'with', 'only', 'haven', 'up', "doesn't", 'of', 'theirs', 'having', 'what', 'wouldn', 'over', 'above', 'there', "should've", 'd', "didn't", 'had', "couldn't", 'than', "hasn't", 'about', "you've", 'himself', 'again', "hadn't", 'between', 'being', 'down'

In [109]:
news['clean_body'] = news['Body'].apply(remove_stop_words) 
news['clean_title'] = news['Title'].apply(remove_stop_words) 
#remove_stop_words_Body

In [116]:
# ## Split data
print("\nSplitting data")
title_tr, title_te, category_tr, category_te = train_test_split(news['clean_body'], news.Category,test_size =.02)
title_tr, title_de, category_tr, category_de = train_test_split(title_tr,category_tr,test_size =.02)
print("Training: ",len(title_tr))
print("Developement: ",len(title_de),)
print("Testing: ",len(title_te))


Splitting data
Training:  2398
Developement:  49
Testing:  50


In [117]:
from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
stop_words = nltk.corpus.stopwords.words("english")
vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

vectorizer.fit(iter(title_tr))
Xtr = vectorizer.transform(iter(title_tr))
Xde = vectorizer.transform(iter(title_de))
Xte = vectorizer.transform(iter(title_te))

encoder = LabelEncoder()
encoder.fit(category_tr)
Ytr = encoder.transform(category_tr)
Yde = encoder.transform(category_de)
Yte = encoder.transform(category_te)


Vectorizing data


In [118]:
Xtr

<2398x16647 sparse matrix of type '<class 'numpy.int64'>'
	with 107201 stored elements in Compressed Sparse Row format>

In [119]:
# ## Feature Reduction
# We can check the variance of the feature and drop them based on a threshold
print("\nApplyting Feature Reduction")
print("Number of features before reduction : ", Xtr.shape[1])
selection = VarianceThreshold(threshold=0.001)
Xtr_whole = copy.deepcopy(Xtr)
Ytr_whole = copy.deepcopy(Ytr)
selection.fit(Xtr)
Xtr = selection.transform(Xtr)
Xde = selection.transform(Xde)
Xte = selection.transform(Xte)
print("Number of features after reduction : ", Xtr.shape[1])


Applyting Feature Reduction
Number of features before reduction :  16647
Number of features after reduction :  7390


In [120]:
# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(Xtr, Ytr)
pred = dc.predict(Xde)



Training baseline classifier


In [121]:
print(classification_report(Yde, pred))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.17      0.12      0.14         8
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         0
          20       0.00      0.00      0.00         0
          24       0.00      0.00      0.00         1
          25       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         2
          32       0.00      0.00      0.00         3
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         2
          40       0.00      0.00      0.00         0
          42       0.00    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [122]:

# ### Decision Tree
print("Training Decision tree")
dt = DecisionTreeClassifier()
dt.fit(Xtr, Ytr)
pred = dt.predict(Xde)
print(classification_report(Yde, pred))

Training Decision tree
              precision    recall  f1-score   support

          10       0.62      0.62      0.62         8
          13       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         0
          20       0.00      0.00      0.00         0
          24       0.50      1.00      0.67         1
          25       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         0
          31       0.00      0.00      0.00         2
          32       0.00      0.00      0.00         3
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         2
          40       0.00      0.00      0.00         0
          42       0.25      1.00      0.40         1
          44       0.00      0.00      0.00         0
          45       0.00      0.00      0.00         1
    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [123]:
# ### Random Forest
print("Training Random Forest")
rf = RandomForestClassifier(n_estimators=40)
rf.fit(Xtr, Ytr)
pred = rf.predict(Xde)
print(classification_report(Yde, pred))

Training Random Forest
              precision    recall  f1-score   support

          10       0.60      0.75      0.67         8
          13       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         0
          24       0.50      1.00      0.67         1
          25       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         2
          32       1.00      0.33      0.50         3
          34       0.00      0.00      0.00         0
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         2
          42       0.00      0.00      0.00         1
          45       0.00      0.00      0.00         1
          46       0.00      0.00      0.00         1
          47       1.00      1.00      1.00         1
          49       0.00      0.00      0.00         2
    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [124]:

# ### Multinomial Naive Bayesian
print("Training Multinomial Naive Bayesian")
nb = MultinomialNB()
nb.fit(Xtr, Ytr)
pred_nb = nb.predict(Xde)
print(classification_report(Yde, pred_nb))


Training Multinomial Naive Bayesian
              precision    recall  f1-score   support

          10       0.70      0.88      0.78         8
          13       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         0
          24       0.50      1.00      0.67         1
          25       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         2
          32       0.75      1.00      0.86         3
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         2
          42       0.50      1.00      0.67         1
          45       0.00      0.00      0.00         1
          46       0.00      0.00      0.00         1
          47       0.50      1.00      0.67         1
          49       0.00      0.00      0.00         2
          50       0.50      1.00      0.67  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [125]:
# ### Support Vector Classification
print("Training Support Vector Classification")
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtr, Ytr)
pred = svc.predict(Xde)
print(classification_report(Yde, pred))


Training Support Vector Classification


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

          10       1.00      0.38      0.55         8
          13       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          24       0.00      0.00      0.00         1
          25       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         2
          32       0.00      0.00      0.00         3
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         2
          42       0.00      0.00      0.00         1
          45       0.00      0.00      0.00         1
          46       0.00      0.00      0.00         1
          47       0.00      0.00      0.00         1
          49       0.00      0.00      0.00         2
          50       0.00      0.00      0.00         1
          52       0.00      0.00      0.00         1
          53       0.00    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [126]:
# ### Multilayered Perceptron
print("Training Multilayered Perceptron")
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(Xtr, Ytr)
pred = mlp.predict(Xde)
print(classification_report(Yde, pred))

Training Multilayered Perceptron
              precision    recall  f1-score   support

          10       0.67      0.75      0.71         8
          11       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          18       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         0
          24       1.00      1.00      1.00         1
          25       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         2
          32       1.00      1.00      1.00         3
          34       0.00      0.00      0.00         0
          37       0.50      1.00      0.67         1
          39       0.00      0.00      0.00         2
          40       0.00      0.00      0.00         0
          42       0.17      1.00      0.29         1
          45       0.00      0.00      0.00         1
          46       0.00      0.00      0.00     

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [127]:
# # Final Model: Multinomial Naive Bayesian
# **Multinomial Naive Bayesian** works the best. Lets run NB on our test data and get the confusion matrix and its heat map.
# ## Predict test data
print("\n\nPredicting test data using Multinomial Naive Bayesian")
pred_final = nb.predict(Xte)
print(classification_report(Yte, pred_final))




Predicting test data using Multinomial Naive Bayesian
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           7       1.00      1.00      1.00         1
          10       0.55      0.75      0.63         8
          18       0.00      0.00      0.00         1
          24       1.00      1.00      1.00         2
          29       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         1
          32       0.50      0.67      0.57         3
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         1
          40       0.40      0.40      0.40         5
          42       0.14      1.00      0.25         1
          44       0.00      0.00      0.00         1
          49       0.00      0.00      0.00         1
          50       0.00      0.00      0.00         1
          51       1.00      1.00      1.00         1
          54       0.25  